<a href="https://colab.research.google.com/github/avesselinov/ds2002s24/blob/main/new_discord_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install discord
%pip install openai
import os
import openai
import pandas as pd
from discord import Intents, Client, Message
from google.colab import drive
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Step 0: Load the Discord token from environment variables
TOKEN = os.getenv('DISCORD_TOKEN')

# Mount Google Drive
drive.mount('/content/drive')

# Load NFL data from CSV
nfl_data_path = "/content/drive/My Drive/nfl.csv"
nfl_data = pd.read_csv(nfl_data_path)

# Step 1: BOT SETUP Without intents your bot won't respond
intents: Intents = Intents.default()
intents.message_content = True
client: Client = Client(intents=intents)

# OpenAI API Key
openai.api_key = "sk-proj-kyzqp2YGLqzGdbx0bIXvT3BlbkFJlvvWH8Fof52HEugDQSPb"

# Define a function to ask OpenAI
def ask_openai(question):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Choose the model you prefer
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question}
        ]
    )
    answer = response['choices'][0]['message']['content']
    return answer

# Define a function to handle user commands
def handle_command(command):
    if command.startswith("!player"):
        player_name = command.split(" ", 1)[1].strip()
        player_data = nfl_data[nfl_data['Player'].str.contains(player_name, case=False)]
        if not player_data.empty:
            return player_data.to_string(index=False)
        else:
            return f"No player found with the name '{player_name}'"
    elif command.startswith("!team"):
        team_name = command.split(" ", 1)[1].strip()
        team_data = nfl_data[nfl_data['Team'].str.contains(team_name, case=False)]
        if not team_data.empty:
            return team_data.to_string(index=False)
        else:
            return f"No team found with the name '{team_name}'"
    else:
        return ask_openai(command)

# Define a function to send message
async def send_message(message: Message, response: str) -> None:
    try:
        await message.channel.send(response)
    except Exception as e:
        print(e)

# Step 3: Handle the startup of the bot
@client.event
async def on_ready() -> None:
    print(f'{client.user} is now running!')

# Step 4: Let's handle the messages
@client.event
async def on_message(message: Message) -> None:
    if message.author == client.user: # The bot wrote the message, or the bot talks to itself
        return

    username: str = str(message.author)
    user_message: str = message.content
    channel: str = str(message.channel)

    print(f'[{channel}] {username}: "{user_message}"')

    # Check if it's a command or a regular message
    if user_message.startswith("!"):
        response = handle_command(user_message)
    elif user_message.lower() == "help":
        response = (
            "!player [player_name]: Get information about a player\n"
            "!team [team_name]: Get information about a team\n"
            "!help: Display this help message"
        )
    else:
        response = ask_openai(user_message)

    await send_message(message, response)

# Step 5: Main starting point
def main1():
    client.run(token=TOKEN)

if __name__ == '__main1__':
    main1()
















# Function to perform data analysis
def analyze_data():
    try:
        # Retrieve data from MongoDB
        collection = db['nfl_data']  # Choose your collection name
        cursor = collection.find({})
        data = list(cursor)
        if len(data) == 0:
            print("No data available for analysis.")
            return

        # Convert data to DataFrame for analysis
        df = pd.DataFrame(data)

        # Check for required columns
        required_columns = ['Yards.Gained', 'Touchdown', 'down', 'GoalToGo', 'posteam', 'DefensiveTeam',
                            'PlayType', 'FieldGoalResult', 'Safety', 'Timeout_Indicator', 'Season']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print("Error: Missing columns in the dataset:", missing_columns)
            return

        # Describe trends over time for specific fields
        print("Trends Over Time:")

        # Number of Touchdowns per Season
        print("\nNumber of Touchdowns per Season:")
        touchdowns_by_season = df[df['Touchdown'] == 1].groupby('Season').size()
        print(touchdowns_by_season)
        # Plot number of touchdowns per season
        plt.figure(figsize=(10, 6))
        touchdowns_by_season.plot(kind='bar', color='lightgreen')
        plt.title('Number of Touchdowns per Season')
        plt.xlabel('Season')
        plt.ylabel('Number of Touchdowns')
        plt.xticks(rotation=45)
        plt.show()

        # Play Types Analysis
        print("\nPlay Types Analysis:")
        play_types = df['PlayType'].value_counts()
        print(play_types)
        # Plot play types
        plt.figure(figsize=(12, 8))
        play_types.plot(kind='bar', color='salmon')
        plt.title('Play Types Distribution')
        plt.xlabel('Play Type')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        plt.show()

        # Field Goal Success Rate
        print("\nField Goal Success Rate:")
        field_goals_made = df[df['FieldGoalResult'] == 'Good'].shape[0]
        field_goals_attempted = df[df['FieldGoalResult'].notnull()].shape[0]
        field_goal_success_rate = (field_goals_made / field_goals_attempted) * 100 if field_goals_attempted != 0 else 0
        print(f"{field_goal_success_rate:.2f}%")

        # Safety Occurrences
        print("\nSafety Occurrences:")
        safety_count = df[df['Safety'] == 1].shape[0]
        print(safety_count)

        # Timeout Analysis
        print("\nTimeout Analysis:")
        timeout_counts = df['Timeout_Indicator'].value_counts()
        print(timeout_counts)
        # Plot timeout indicators
        plt.figure(figsize=(8, 6))
        timeout_counts.plot(kind='bar', color='lightblue')
        plt.title('Timeout Indicators')
        plt.xlabel('Timeout Indicator')
        plt.ylabel('Count')
        plt.xticks(rotation=0)
        plt.show()

        # Average Yards Gained per Play Type
        print("\nAverage Yards Gained per Play Type:")
        avg_yards_by_play_type = df.groupby('PlayType')['Yards.Gained'].mean().sort_values(ascending=False)
        print(avg_yards_by_play_type)
        # Plot average yards gained per play type
        plt.figure(figsize=(12, 8))
        avg_yards_by_play_type.plot(kind='bar', color='orange')
        plt.title('Average Yards Gained per Play Type')
        plt.xlabel('Play Type')
        plt.ylabel('Average Yards Gained')
        plt.xticks(rotation=45, ha='right')
        plt.show()

        # Most Frequent Defensive Teams
        print("\nMost Frequent Defensive Teams:")
        top_defensive_teams = df['DefensiveTeam'].value_counts().head(10)
        print(top_defensive_teams)
        # Plot most frequent defensive teams
        plt.figure(figsize=(10, 6))
        top_defensive_teams.plot(kind='bar', color='lightcoral')
        plt.title('Top 10 Most Frequent Defensive Teams')
        plt.xlabel('Defensive Team')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.show()

        # Explanation of changes
        print("\nExplanation of Changes:")
        print("The number of touchdowns per season provides insights into the scoring trends over the years.")
        print("Play types distribution reveals the variety of strategies teams employ during games.")
        print("Field goal success rate shows how well teams convert field goal attempts.")
        print("Safety occurrences and timeout indicators provide additional context about game situations and strategies.")
        print("Average yards gained per play type gives an idea of the effectiveness of different play strategies.")
        print("The most frequent defensive teams highlight the teams that appear most often in the dataset.")

    except Exception as e:
        print("Error analyzing data:", e)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-a763a45402a1>:21: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv(nfl_data_path)
